In [1]:
from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing import image
import numpy as np
import cv2
import os

# Load model (without top layer)
model = Xception(weights='imagenet', include_top=False, pooling='avg')


In [2]:
def extract_frames(video_path, max_frames=10):
    cap = cv2.VideoCapture(video_path)
    frames = []
    count = 0
    while cap.isOpened() and count < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
        # Resize + convert to array
        frame = cv2.resize(frame, (299, 299))
        img_array = image.img_to_array(frame)
        img_array = preprocess_input(img_array)
        frames.append(img_array)
        count += 1
    cap.release()
    return np.array(frames)


In [3]:
def is_deepfake(video_path):
    frames = extract_frames(video_path)
    if len(frames) == 0:
        return "Invalid or corrupted video"

    # Extract features using Xception
    features = model.predict(frames, verbose=0)
    avg_feature = np.mean(features, axis=0)

    # Dummy classifier: If average feature value > threshold → Fake
    # (In real-world, train an actual classifier like SVM or NN)
    fake_score = np.mean(avg_feature)
    print(f"Fake Score: {fake_score:.4f}")
    return "FAKE" if fake_score > 0.55 else "REAL"


In [4]:
video_path = 'sample_video.mp4'
result = is_deepfake(video_path)
print("Prediction:", result)


Fake Score: 0.1279
Prediction: REAL


In [6]:
import cv2
import random

# Load Haar Cascade
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Load the video
cap = cv2.VideoCapture('sample_video.mp4')

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Resize (optional): for performance
    frame = cv2.resize(frame, (640, 480))

    # Convert to gray for detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        # Simulate fake score (for now)
        fake_score = round(random.uniform(0, 1), 2)
        prediction = "FAKE" if fake_score > 0.5 else "REAL"

        # Box color
        box_color = (0, 0, 255) if prediction == "FAKE" else (0, 255, 0)

        # Draw face box
        cv2.rectangle(frame, (x, y), (x + w, y + h), box_color, 2)

        # Texts
        label1 = f"Prediction: {prediction}"
        label2 = f"Fake Score: {fake_score}"

        # Text background box
        cv2.rectangle(frame, (x, y - 40), (x + w, y), box_color, -1)

        # Draw texts
        cv2.putText(frame, label1, (x + 5, y - 25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
        cv2.putText(frame, label2, (x + 5, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)

    # Show frame
    cv2.imshow("🧠 Deepfake Detection", frame)

    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


import cv2
import random

# Load Haar cascade
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Load video
cap = cv2.VideoCapture('sample_video.mp4')

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        # Draw rectangle
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Simulate Fake Score (random between 0 to 1)
        fake_score = round(random.uniform(0, 1), 2)

        # Determine prediction
        prediction = "FAKE" if fake_score > 0.5 else "REAL"

        # Draw texts
        text1 = f"Prediction: {prediction}"
        text2 = f"Fake Score: {fake_score}"

        # Draw background rectangle for text
        cv2.rectangle(frame, (x, y - 40), (x + w, y), (0, 255, 0), -1)

        # Put text on the video
        cv2.putText(frame, text1, (x + 5, y - 25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
        cv2.putText(frame, text2, (x + 5, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)

    # Show video frame
    cv2.imshow("Deepfake Detection", frame)

    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import os
import numpy as np
import cv2
from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Load feature extractor
feature_extractor = Xception(weights='imagenet', include_top=False, pooling='avg')

def extract_frames(video_path, max_frames=10):
    cap = cv2.VideoCapture(video_path)
    frames = []
    count = 0
    while cap.isOpened() and count < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (299, 299))
        img_array = image.img_to_array(frame)
        img_array = preprocess_input(img_array)
        frames.append(img_array)
        count += 1
    cap.release()
    return np.array(frames)

def extract_feature_from_video(video_path):
    frames = extract_frames(video_path)
    if len(frames) == 0:
        return None
    features = feature_extractor.predict(frames, verbose=0)
    avg_feature = np.mean(features, axis=0)
    return avg_feature

# Load dataset
def load_dataset(dataset_dir):
    features = []
    labels = []
    for label_folder, label in [('real', 0), ('fake', 1)]:
        folder_path = os.path.join(dataset_dir, label_folder)
        for file in os.listdir(folder_path):
            video_path = os.path.join(folder_path, file)
            feat = extract_feature_from_video(video_path)
            if feat is not None:
                features.append(feat)
                labels.append(label)
    return np.array(features), np.array(labels)

# Load features and labels
X, y = load_dataset("dataset")
print("Dataset loaded:", X.shape, y.shape)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build classifier model
model = Sequential([
    Dense(512, activation='relu', input_shape=(X.shape[1],)),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')  # Binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train model
model.fit(X_train, y_train, epochs=10, batch_size=8, validation_split=0.2)

# Evaluate
y_pred = (model.predict(X_test) > 0.5).astype(int)
print(classification_report(y_test, y_pred))

# Save the model
model.save("deepfake_classifier.h5")
